# 示例选择器
- 根据长度动态选择
- 根据语义相似度动态选择
- 使用最大边际相关性进行选择

In [15]:
!pip install cohere langchain-cohere langchain-community -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


# 根据长度动态选择

In [1]:
# 示例：通过计算最终长度，根据输入提示的长度智能地拦截或添加提示。
# Example of Intelligently intercepting or adding prompts by calculating the final length based on the length of the input prompts.

from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# --- 1. 定义示例集 (EXAMPLES) ---

# 假设这里有很多提示示例
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

# --- 2. 构建示例模板 (EXAMPLE PROMPT TEMPLATE) ---

# 构建单个示例的格式模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n反义词: {output}",
)

# --- 3. 初始化长度示例选择器 (LENGTH BASED EXAMPLE SELECTOR) ---

# 调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    # 传入提示示例组
    examples=examples,
    # 传入示例模板
    example_prompt=example_prompt,
    # 设置格式化后的提示的最大长度。如果提示总长超过 25 个“词”，将开始删除示例。
    max_length=25,
    # 内置get_text_length, 如果默认分词计算方式不满足，可以自己扩展
    # 这里我们使用一个简单的基于空格的分词函数来计算长度（词数）。
    get_text_length=lambda x: len(x.split()),
)

# --- 4. 构建动态 FewShot 提示模板 (FEWSHOT PROMPT TEMPLATE) ---

# 使用小样本提示模板实现动态示例的调用
dynamic_prompt = FewShotPromptTemplate(
    # 示例选择器
    example_selector=example_selector,
    # 示例提示模板
    example_prompt=example_prompt,
    # 前缀（Instruction）
    prefix="给出给定的输入的反义词",
    # 后缀，包含用户输入变量
    suffix="形容词: {adjective}\n反义词",
    # 传入输入变量
    input_variables=["adjective"]
)

In [2]:
# --- 5. 运行示例 (EXECUTION) ---

# 正常长度输入 (短字符串): 应该包含所有或大部分示例，因为总长度不会超过 max_length=25
print("--- 正常长度输入 (应包含多个示例) ---")
short_string = "small"
print(dynamic_prompt.format(adjective=short_string))


# 过长输入：如果输入长度很长，最终输出将根据长度要求减少示例数量
print("\n--- 过长输入 (应减少示例数量) ---")
# 这是一个非常长的字符串，它会占用很多“词数”
long_string = "big and huge adh massive and large and gigantic and tall and much much much much much bigger then everyone"
# 由于输入 'long_string' 太长，它会挤占空间，导致示例被大量删除或全部删除
print(dynamic_prompt.format(adjective=long_string))

# 可以看到，当输入越长时，为了满足 max_length 的限制，示例的数量就会动态减少。

--- 正常长度输入 (应包含多个示例) ---
给出给定的输入的反义词

输入: happy
反义词: sad

输入: tall
反义词: short

输入: sunny
反义词: gloomy

输入: windy
反义词: calm

输入: 高兴
反义词: 悲伤

形容词: small
反义词

--- 过长输入 (应减少示例数量) ---
给出给定的输入的反义词

输入: happy
反义词: sad

形容词: big and huge adh massive and large and gigantic and tall and much much much much much bigger then everyone
反义词


# 根据输入的语义相似度动态选择
- 筛选示例组中与输入的语义相似度最高的示例
- 本质：将问题与示例嵌入向量空间后进行搜索对比
- 依赖：向量数据库

In [3]:
# Use the maximum cosine similarity to retrieve relevant examples to make the examples as close as possible to the input
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

import os

# 1. Define the base prompt template for a single example
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n反义: {output}",
)

# 2. Define the examples for the few-shot learning
# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"}, # Note: Corrected "cate" to "calm" based on context
]

In [8]:
# 3. Create the Example Selector using Semantic Similarity
# Pass in the example group.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # Pass in the example group.
    examples,
    # Use hugging face embeddings for similarity search
    HuggingFaceEmbeddings(model_name="moka-ai/m3e-base"),
    # Use the Chroma vector database to implement the process storage of similar results
    FAISS,
    # Number of results
    k=1,
)

# 4. Create the final Few-Shot Prompt Template
# Pass in the selector and template, as well as the prefix and suffix and input variables
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出一个输入词的反义词。",
    suffix="输入: {adjective}\n反义:",
    input_variables=["adjective"],
)

# 5. Test the prompt by passing in a new adjective "worried"
# Pass in a new adjective, expected output is happy/sad
print(similar_prompt.format(adjective="worried"))

C:\Users\jz292\.conda\envs\langchain-learning\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jz292\.cache\huggingface\hub\models--moka-ai--m3e-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular H

给出一个输入词的反义词。

输入: energetic
反义: lethargic

输入: worried
反义:


# 使用最大边际相关性动态选择示例（MMR）
- 筛选示例组中符合MMR规则的示例
- 本质：将问题与示例嵌入向量空间后进行搜索比对
- 依赖:向量数据库
- MMR：是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡。MMR会首先找出与输入最相似（即余弦相似度最大）的样本。然后在迭代添加样本的过程中，对于与已选择样本过于接近（即相似度过高）的样本惩罚。MMR既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性。关注如何在相关性和多样性之间找到一个平衡。

In [9]:
import os
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

# 设置 API 密钥和基础 URL
# 假设您已在环境变量中设置了这些值
# api_base = os.getenv("OPENAI_API_BASE")
# api_key = os.getenv("OPENAI_API_KEY")

# --- 1. 示例数据 ---
# 假设有许多 prompt 示例 (输入为形容词，输出为情绪)
examples = [
    {"input": "happy", "output": "joy"},
    {"input": "sad", "output": "grief"},
    {"input": "angry", "output": "fury"},
    {"input": "calm", "output": "peace"},
    {"input": "anxious", "output": "fear"}, # 新增一个示例
    {"input": "gloomy", "output": "sadness"}, # 新增一个示例
    {"input": "excited", "output": "anticipation"}, # 新增一个示例
    {"input": "bored", "output": "apathy"}, # 新增一个示例
]

# 构造单个示例的 prompt 模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)


# --- 2. 创建示例选择器 (MMR) ---

# 调用 MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 传入示例列表
    examples,
    # 使用 OpenAI 的 embedding 进行相似性搜索
    # 假设您已经配置了 OPENAI_API_BASE 和 OPENAI_API_KEY
    HuggingFaceEmbeddings(model_name="moka-ai/m3e-base"),
    # 设置使用 FAISS 作为向量数据库
    vectorstore_cls=FAISS,
    # 返回结果的数量
    k=2
)

# --- 3. 创建 FewShotPromptTemplate ---

# MMR Prompt 模板
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    # prompt 的前缀
    prefix="你是一个情感分析模型，请根据输入的形容词，输出最符合的情绪。",
    # prompt 的后缀/新的输入
    suffix="\n请根据上面的示例，为下一个形容词找到对应的情绪：\n输入: {adjective}\n输出:",
    # 新的输入变量名
    input_variables=["adjective"],
)

# --- 4. 测试 ---

# 当我们输入一个描述情绪的词时，它应该选择最相似的两个示例来填充 prompt
# 比如输入 'terrified' (极度恐惧), 应该会选择 'anxious' 和 'angry' (取决于 embedding 的相似度计算)
print("--- 针对 'terrified' 的完整 prompt ---")
print(mmr_prompt.format(adjective="terrified"))

# 再试一个 'joyful' (快乐的)
print("\n--- 针对 'joyful' 的完整 prompt ---")
print(mmr_prompt.format(adjective="joyful"))

--- 针对 'terrified' 的完整 prompt ---
你是一个情感分析模型，请根据输入的形容词，输出最符合的情绪。

输入: anxious
输出: fear

输入: sad
输出: grief


请根据上面的示例，为下一个形容词找到对应的情绪：
输入: terrified
输出:

--- 针对 'joyful' 的完整 prompt ---
你是一个情感分析模型，请根据输入的形容词，输出最符合的情绪。

输入: happy
输出: joy

输入: anxious
输出: fear


请根据上面的示例，为下一个形容词找到对应的情绪：
输入: joyful
输出:
